In [23]:
import numpy as np
from random import choice
import random
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from PIL import Image
import pandas as pd
from numpy import asarray
import cv2

import time


In [24]:


'''
A function that checks for adjacent cells and also ensures that they are within the 
boundaries of the matrix
'''
def neighbours(rows,columns,start_pt):

    x1y1 = np.array([(0,-1),(0,1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]) # possible steps in 8 directions
    x2y2 = list(map(tuple,x1y1 + start_pt))
    z = []
    for i in range(len(x2y2)):
        if( 0<= x2y2[i][0] <= columns-1 and 0<= x2y2[i][1] <= rows-1):
            z.append((x2y2[i]))  #getting just the possible coordinates around a pixel
        else:
            continue
    z = list(map(tuple,z))
    
    return z

'''
Edge detection filter to determine the number of pixels other than zero, for a 5x5 sobel filter

'''

def edge(array):
    sobelxy = cv2.Sobel(src=array, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5)
    p = np.count_nonzero(sobelxy != 0)

    return p

'''
Function to count the adjacent zero values around each pixel of value 1
'''
def zero(array):
    
    shp = array.shape
    
    zeros_around = 0

    for i in range(shp[0]-1):
        for j in range(shp[0]-1):
            if array[i,j] == 1:
                points_around =  neighbours(shp[0], shp[1], (i,j))
                zeros = list(filter(lambda x : array[x] == 0, points_around))
                zeros_around+= len(zeros)
    
    
    return zeros_around




'''
This function takes number of rows, columns and no of particles as iterations, stickiness to 
simulate DLA

'''
def DLA(rows,columns,itrn,k):
    data = []
    start_time = time.time()
    arr = np.zeros((rows,columns))
    arr[int((rows-1)/2)+1][int((columns-1)/2)+1] = 1
   
    for i in range(itrn): 
        ran_indx = random.randrange(0,columns)
        
        #chooses a point on the edges of the square matrix
        start_pt = choice([(ran_indx,0),(ran_indx,columns-1),(0,ran_indx),(rows-1,ran_indx)])
        z = neighbours(rows,columns,start_pt)
        
        #checks for if any of neighbouring pixels are 1 and also the random integer between 0 and 1
        # is less than the stickiness or not
       
        while(not (any(map(lambda x:arr[x] == 1, z)) and np.random.rand()< k)):    
            adj_pts = list(filter(lambda x : arr[x] == 0, z))
            s = np.random.randint(len(adj_pts))    
            start_pt = adj_pts[s]
            z = neighbours(rows,columns,start_pt)
            
            
        arr[start_pt] = 1
        if((i+1)%100 == 0):  #For everytime I am saving the data for adding 100 particles
            name = f'DLA_101_' + str(i+1) + '_' + str(k)+'.png'
            matplotlib.image.imsave(name, 1-arr)
            p = edge(arr)
            print("positive_pixels_after_edge_detection: ", p)
            ze = zero(arr)
            print("Total zeros around each pixel of value 1 : ", ze)
            data.append([i+1, k, p, ze, (time.time() - start_time),rows ])
            print(i, "--- %s seconds ---" % (time.time() - start_time))
         
    df = pd.DataFrame(data, columns=['N', 'k', 'edge', 'zeros_around', 'time_sec','square_matrix_size']) 
        
    return arr,df

In [ ]:

#This will generate DLA images with a step size of 100 till 1500 with all the necessary data 

for i in np.linspace(0.001, 0.005, 20):
             a = DLA(101,101,1500,i)
             df = a[1]
             df.to_csv(f'data_k = '+str(i)+'.csv')
        
       
      

positive_pixels_after_edge_detection:  331
Total zeros around each pixel of value 1 :  462
99 --- 1688.157222032547 seconds ---
positive_pixels_after_edge_detection:  533
Total zeros around each pixel of value 1 :  804
199 --- 2461.4255254268646 seconds ---
positive_pixels_after_edge_detection:  708
Total zeros around each pixel of value 1 :  1150
299 --- 3120.216574907303 seconds ---
positive_pixels_after_edge_detection:  856
Total zeros around each pixel of value 1 :  1418
399 --- 3740.840943813324 seconds ---
positive_pixels_after_edge_detection:  1036
Total zeros around each pixel of value 1 :  1686
499 --- 4205.218238353729 seconds ---
positive_pixels_after_edge_detection:  1209
Total zeros around each pixel of value 1 :  2022
599 --- 4649.949274301529 seconds ---
positive_pixels_after_edge_detection:  1367
Total zeros around each pixel of value 1 :  2318
699 --- 4937.676241874695 seconds ---
positive_pixels_after_edge_detection:  1562
Total zeros around each pixel of value 1 :  2